# Collectl Mem Log Analysis

## Functionalities
- Plot memory utilization graphs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    collectl.tar.gz
  ...
  [node-n]/
    collectl.tar.gz
```

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_[TIMESTAMP]"

########## MEMORY
# Analyzed metric (options: "used", "free", "slab", "mapped", "anon", "anonh", "inactive", "hits")
COLLECTL_MEM_METRIC = "free"

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from utils.utils import *

experiment_dirpath = os.path.join(os.path.abspath(""), "..", "data", EXPERIMENT_DIRNAME)

## Log Parsing & Processing

In [ ]:
# Build data frames
mem = pd.concat([df[2] for df in get_collectl_mem_df(experiment_dirpath)])

In [ ]:
# Extract experiment information
start_time = get_experiment_start_time(experiment_dirpath)
node_names = get_node_names(experiment_dirpath)

In [ ]:
# (Re) Build columns
mem["timestamp"] = mem.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
mem["window_1000"] = mem["timestamp"].round(0).multiply(1000)

In [ ]:
# (Re) Create index
mem.set_index("timestamp", inplace=True)
mem.sort_index(inplace=True)

## Memory Monitoring

In [ ]:
fig = plt.figure(figsize=(24, len(node_names) * 12))
for (i, node_name) in enumerate(sorted(node_names)):
    # Data frame
    df = mem[(mem["node_name"] == node_name)].groupby(["window_1000"])[COLLECTL_MEM_METRIC].mean()
    # Plot
    ax = fig.add_subplot(len(node_names), 1, i + 1)
    ax.set_xlim((0, df.index.max()))
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="line", title="%s - Mem Utilization" % node_name, xlabel="Time (millisec)", ylabel="%s" % COLLECTL_MEM_METRIC, grid=True)

In [ ]:
########## ZOOM IN
# Minimum time (in sec)
MIN_TIME = None
# Maximum time (in sec)
MAX_TIME = None

if MIN_TIME and MAX_TIME:
    fig = plt.figure(figsize=(24, len(node_names) * 12))
    for (i, node_name) in enumerate(sorted(node_names)):
        # Data frame
        df = mem[(mem["node_name"] == node_name) & (mem.index >= MIN_TIME) & (mem.index <= MAX_TIME)].groupby(["timestamp", "hw_no"])[COLLECTL_MEM_METRIC].mean().unstack()
        # Plot
        ax = fig.add_subplot(len(node_names), 1, i + 1)
        ax.set_xlim((df.index.min(), df.index.max()))
        ax.grid(alpha=0.75)
        df.plot(ax=ax, kind="line", title="%s - Mem Utilization" % node_name, xlabel="Time (millisec)", ylabel="%s" % COLLECTL_MEM_METRIC, grid=True, legend=True)